In [28]:
#librerias
import sys
from awsglue.context import GlueContext
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql import functions as F
from pyspark.sql.types import BooleanType, TimestampType, IntegerType
from pyspark.ml.feature import Tokenizer
import pyspark.pandas as ps
import string
import boto3
import os
from collections import Counter
import re
from awsglue.job import Job

In [29]:
#Crear conexiones
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
logger = glueContext.get_logger()

In [30]:
#Parametros y Funciones
s3_client = boto3.client('s3')
bucket_name = 'nequijmmarinq'
file_key = 'stopwords'
input_path = 's3://nequijmmarinq/twcs.csv'
output_path_clean = "s3://nequijmmarinq/processed/clean_data/"

PUNCT_TO_REMOVE = string.punctuation
EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}

chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""
def remove_punctuation(text):
    text = str(text)
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

def remove_stopwords(text):
    text = str(text)
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

def remove_freqwords(text):
    text = str(text)
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

def remove_rarewords(text):
    text = str(text)
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

def remove_emoji(string):
    text = str(string)
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_emoticons(text):
    text = str(text)
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

def remove_urls(text):
    text = str(text)
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_html(text):
    text = str(text)
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def chat_words_conversion(text):
    text = str(text)
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [31]:
#Leer el archivo en spark 
logger.info("Leyendo datos desde S3...")
df = spark.read.option("header", True).csv(input_path)

In [32]:
df.schema

StructType([StructField('tweet_id', StringType(), True), StructField('author_id', StringType(), True), StructField('inbound', StringType(), True), StructField('created_at', StringType(), True), StructField('text', StringType(), True), StructField('response_tweet_id', StringType(), True), StructField('in_response_to_tweet_id', StringType(), True)])


In [33]:
df.show()

+--------+------------+-------+--------------------+--------------------+-----------------+-----------------------+
|tweet_id|   author_id|inbound|          created_at|                text|response_tweet_id|in_response_to_tweet_id|
+--------+------------+-------+--------------------+--------------------+-----------------+-----------------------+
|       1|  sprintcare|  False|Tue Oct 31 22:10:...|@115712 I underst...|                2|                      3|
|       2|      115712|   True|Tue Oct 31 22:11:...|@sprintcare and h...|             null|                      1|
|       3|      115712|   True|Tue Oct 31 22:08:...|@sprintcare I hav...|                1|                      4|
|       4|  sprintcare|  False|Tue Oct 31 21:54:...|@115712 Please se...|                3|                      5|
|       5|      115712|   True|Tue Oct 31 21:49:...|  @sprintcare I did.|                4|                      6|
|       6|  sprintcare|  False|Tue Oct 31 21:46:...|@115712 Can you p...

In [34]:
#Eliminar registros incompletos y variables innecesarias
logger.info("Eliminar Registros Incompletos y variables innecesarias...")
df_clean = (
    df.dropna(subset=["text", "created_at"])  # Eliminar registros incompletos
      .withColumn("inbound", F.col("inbound").cast(BooleanType()))
      .withColumn("inbound", F.col("inbound").cast(IntegerType()))
      .drop("tweet_id", "author_id", "response_tweet_id", "in_response_to_tweet_id")  # columnas innecesarias
)

In [35]:
df_clean.schema

StructType([StructField('inbound', IntegerType(), True), StructField('created_at', StringType(), True), StructField('text', StringType(), True)])


In [36]:
df_clean.count()

2812943


In [37]:
logger.info("Validación de calidad de los datos...")
null_counts = df_clean.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_clean.columns
])
null_counts.show()

logger.info(f"Total registros después de limpieza: {df_clean.count()}")


+-------+----------+----+
|inbound|created_at|text|
+-------+----------+----+
|   1167|         0|   0|
+-------+----------+----+


In [38]:
df_clean.show()

+-------+--------------------+--------------------+
|inbound|          created_at|                text|
+-------+--------------------+--------------------+
|      0|Tue Oct 31 22:10:...|@115712 I underst...|
|      1|Tue Oct 31 22:11:...|@sprintcare and h...|
|      1|Tue Oct 31 22:08:...|@sprintcare I hav...|
|      0|Tue Oct 31 21:54:...|@115712 Please se...|
|      1|Tue Oct 31 21:49:...|  @sprintcare I did.|
|      0|Tue Oct 31 21:46:...|@115712 Can you p...|
|      1|Tue Oct 31 21:45:...|@sprintcare is th...|
|      0|Tue Oct 31 22:10:...|@115713 This is s...|
|      1|Tue Oct 31 22:04:...|@sprintcare You g...|
|      0|Tue Oct 31 20:03:...|@115713 We unders...|
|      1|Tue Oct 31 20:00:...|@sprintcare Since...|
|      0|Tue Oct 31 19:59:...|@115713 H there! ...|
|      1|Tue Oct 31 19:56:...|@115714 y’all lie...|
|      0|Tue Oct 31 22:10:...|@115715 Please se...|
|      1|Tue Oct 31 22:03:...|@115714 whenever ...|
|      0|Tue Oct 31 22:14:...|@115716 What info...|
|      1|Tue

In [39]:
df_clean = df_clean.dropna(subset=["text", "created_at","inbound"])

In [40]:
df_clean.count()

2811776


In [11]:
df_clean.show()

+-------+--------------------+--------------------+
|inbound|          created_at|                text|
+-------+--------------------+--------------------+
|  false|Tue Oct 31 22:10:...|@115712 I underst...|
|   true|Tue Oct 31 22:11:...|@sprintcare and h...|
|   true|Tue Oct 31 22:08:...|@sprintcare I hav...|
|  false|Tue Oct 31 21:54:...|@115712 Please se...|
|   true|Tue Oct 31 21:49:...|  @sprintcare I did.|
|  false|Tue Oct 31 21:46:...|@115712 Can you p...|
|   true|Tue Oct 31 21:45:...|@sprintcare is th...|
|  false|Tue Oct 31 22:10:...|@115713 This is s...|
|   true|Tue Oct 31 22:04:...|@sprintcare You g...|
|  false|Tue Oct 31 20:03:...|@115713 We unders...|
|   true|Tue Oct 31 20:00:...|@sprintcare Since...|
|  false|Tue Oct 31 19:59:...|@115713 H there! ...|
|   true|Tue Oct 31 19:56:...|@115714 y’all lie...|
|  false|Tue Oct 31 22:10:...|@115715 Please se...|
|   true|Tue Oct 31 22:03:...|@115714 whenever ...|
|  false|Tue Oct 31 22:14:...|@115716 What info...|
|   true|Tue

In [12]:
#Realizaremos la limpieza en pandas 
logger.info("Iniciando limpieza de datos...")
df_p = ps.DataFrame(df_clean)
#Aplicar minusculas al texto, ya que disminuye los duplicados
df_p["text"] = df_p["text"].str.lower()

In [13]:
#Remover puntuacion, ya que disminuye los duplicados
df_p["text"] = df_p["text"].apply(lambda text: remove_punctuation(text))

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [14]:
#Se sube el stopwords a s3, se van a eliminar las stopwords, palabras que no aportan mucho al analisis
s3_client.download_file(bucket_name, file_key, '/tmp/stopwords.txt')
with open('/tmp/stopwords.txt', 'r') as file:
    stop_words = file.readlines()
stop_words = [word.strip() for word in stop_words]
STOPWORDS = set(stop_words)
df_p["text"] = df_p["text"].apply(lambda text: remove_stopwords(text))

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [15]:
#Eliminar las 10 palabras mas frecuentes, ya que no aporta tampoco tanto a la información
cnt = Counter()
for text in df_p["text"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)
        
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
df_p["text"] = df_p["text"].apply(lambda text: remove_freqwords(text))

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/pandas/generic.py:647: UserWarning: We recommend using `Series.to_numpy()` instead.
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [16]:
cnt.most_common(10)

[('us', 446024), ('please', 395713), ('dm', 332255), ('help', 263310), ('hi', 223666), ('thanks', 202524), ('get', 196690), ('sorry', 191257), ('like', 144306), ('know', 142810)]


In [17]:
#Eliminar igualmente lo que se conoce como palabras raras o menos frecuentes
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
df_p["text"] = df_p["text"].apply(lambda text: remove_rarewords(text))

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [18]:
#Eliminar Emojis, los emojis ya hacen parte de la vida diaria, sin embargo para el modelamiento no aportan mucho
df_p["text"] = df_p["text"].apply(lambda text: remove_emoji(text))

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [19]:
remove_emoji("game is on 🔥🔥")

'game is on '


In [20]:
#Eliminar emoticones, los emoticones ya hacen parte de la vida diaria, sin embargo para el modelamiento no aportan mucho
df_p["text"] = df_p["text"].apply(lambda text: remove_emoticons(text))

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [21]:
remove_emoticons("Hello :-)")

'Hello '


In [22]:
#Eliminar las urls
df_p["text"] = df_p["text"].apply(lambda text: remove_urls(text))

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [23]:
text = "Driverless AI NLP blog post on https://www.h2o.ai/blog/detecting-sarcasm-is-difficult-but-ai-may-have-an-answer/"
remove_urls(text)

'Driverless AI NLP blog post on '


In [24]:
#Eliminar los tags
df_p["text"] = df_p["text"].apply(lambda text: remove_html(text))

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [24]:
text = """<div>
<h1> H2O</h1>
<p> AutoML</p>
<a href="https://www.h2o.ai/products/h2o-driverless-ai/"> Driverless AI</a>
</div>"""

print(remove_html(text))


 H2O
 AutoML
 Driverless AI


In [25]:
#Conversiones de tipificaciones de jerga de la lengua
chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

df_p["text"] = df_p["text"].apply(lambda text: chat_words_conversion(text))

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [26]:
chat_words_conversion("one minute BRB")

'one minute Be Right Back'


In [ ]:
#logger.info("Tokenizando texto...")
#tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
#df_clean = tokenizer.transform(df_clean)

In [45]:
spark_df.write.mode("overwrite").parquet(output_path_clean)